- target:
- observation log:
- other notes
- check timerange in example

# 0. Defining Variables

In [ ]:
# filing
filename = '43_039_12NOV2022_GSB.LTA_RRLL.RRLLFITS.ms'
homeDir = 'Volumes/G-RAID_Thunderbold_4/xiao_data/GMRT'
workDir = '43_039_12NOV2022_GSB'
plotDir = 'plots/'
calDir = 'calibration/' + workDir + '.'
filenameCal = workDir + '_calibrated.ms'

# fields
ampCal = '3C286'  		# visibility calibrator
gainCal = 'J1925+2106' 	  # (unsure) near target gain calibrator
polCal = ''   	# polarization calibrator
target = 'GRB221009A'

# calibration
antposTable = calDir + 'antpos'
phaseTableAll = calDir + 'G0all'
phaseTable = calDir + 'G0' # only uses the bandpass calibrator
delayTable = calDir + 'K0'
bandpassTable = calDir + 'B0'
gainTable = calDir + 'G1'
crossDelayTable = calDir + 'Kcross'
leakageTable0 = calDir + 'D1'
leakageTable = calDir + 'D2'
polPhaseTable = calDir + 'X1'
fluxTable = calDir + 'fluxscale1'

# msmd.open(filename) # https://casa.nrao.edu/doxygen/classcasacore_1_1MSMetaData.html
# antennas = msmd.antennanames()
refAnt = 'C09'
badAntenna = 'C03,E06'
# goodAntenna = [ant for ant in antennas if ant not in badAntenna.split(',')] # not needed
model = 'calibratorName_bandLetter.im'
channels = 256
spwMid = '0:'+str(int(channels*0.45))+'~'+str(int(channels*0.55))
spwWide = '0:'+str(int(channels*0.08))+'~'+str(int(channels*0.92))
timeRange = '59895.26921727792~59895.27853796347' # time range of ampCal scan
sampleTime = '8.05' # integration time for individual scans
from numpy import log,pi,radians,cos,sin
# i0 = 15.10931396 # Jy, Stokes I in channel 0 of spw 0
# refFreq0 = '4536.0MHz' # frequency of channel 0 of spw 0, from listobs()
# c0 = 0.112 # fractional polarization
# d0 = radians(33) # polarization angle in degrees to radians
# i1 = 7.53174 # Jy, lowest intensity returned by setjy()
# refFreq1 = '1408.935Mhz' # highest frequency returned by setjy()
# baseline0 = 'ea01&ea02'

# plotting, can save in a plot directory
antennaLayoutPlot = plotDir + 'antenna_layout.png'
ampTimePlot = plotDir + 'amplitude_vs_time_all.png'
datastream2 = plotDir + 'datastream_ea01_antenna2.png'
gainphaseRLTimeAllPlot = plotDir + 'gain_phase_RL_vs_time_all.png'
gainphaseRLTimePlot = plotDir + 'gain_phase_RL_vs_time.png'
delayAntennaPlot = plotDir + 'delay_vs_antenna_ea21.png'
bandpassGainAmpChannelPlot = plotDir + 'bandpass_gain_amplitude_vs_channel.png'
bandpassGainPhaseChannelPlot = plotDir + 'bandpass_gain_phase_vs_channel.png'
gainPhaseVsTimePlot = plotDir + 'gain_phase_vs_time.png'
gainAmpVsTimePlot = plotDir + 'gain_amplitude_vs_time.png'
complexPolarizationRatioPlot = plotDir + 'complex_polarization_ratio.png'
modelRRPolAmpVsChannelPlot = plotDir + 'model_RR_pol_amplitude_vs_channel.png'
modelRLAmpVsChannelPlot = plotDir + 'model_RL_pol_amplitude_vs_channel.png'
modelRLPhaseVsChannelPlot = plotDir + 'model_RL_pol_phase_vs_channel.png'
crossDelayTablePlot = plotDir + 'cross_delay_vs_antenna.png'
leakageAmpVsChannelPlot = plotDir + 'leakage_amplitude_vs_channel.png'
leakagePhaseVsChannelPlot = plotDir + 'leakage_phase_vs_channel.png'
leakageAmpVsAntennaPlot0 = plotDir + 'leakage_phase_vs_antenna_prelim.png'
leakageAmpVsAntennaPlot = plotDir + 'leakage_phase_vs_antenna.png'
polGainPhaseVsChannelPlot = plotDir + 'pol_gain_phase_vs_channel.png'
ampCalCorrectedAmpVsChannelPlot = plotDir + 'amplitude_calibrator_corrected_amplitude_vs_channel.png'
ampCalCorrectedPhaseVsChannelPlot = plotDir + 'amplitude_calibrator_corrected_phase_vs_channel.png'
gainCalCorrectedAmpVsChannelPlot = plotDir + 'gain_calibrator_corrected_amplitude_vs_channel.png'
gainCalCorrectedPhaseVsChannelPlot = plotDir + 'gain_calibrator_corrected_phase_vs_channel.png'
targetCorrectedAmpVsBaselinePlot = plotDir + 'target_corrected_amplitude_vs_baseline.png'
cleanImage = 'clean_image'
gridr = 6
gridc = 5
w = gridc * 300
h = gridr * 230
# avgTime = '3543.349109649658' # compare to prev

# 1. Obtaining data

In [ ]:
# navigate to root directory
cd /Volumes/G-RAID_Thunderbold_4/xiao_data

# make sub directory for working in
mkdir workDir

# navigate to work directory
cd workDir

# make sub directory for plots
mkdir plots

# make sub directory for calibration tables
mkdir calibration

In [ ]:
# to handle FTP files

# switch to bash shell within work directory
bash

# wget downloads files from the web
# -c: continue download where left off
# --progress=bar:force --show-progress: show progress bar
# --tries=20 --waitretry=10: try up to 20 times if an error occurs with 10 s between tries
wget -c --progress=bar:force --show-progress --tries=20 --waitretry=10 ftp://naps.ncra.tifr.res.in/* --ftp-user DLFGOEAK --ftp-password DLFGOEAK
# retry command after completion, it should recognize that all files are already downloaded

In [ ]:
# to handle a tarzip in the download folder

# move file from download folder into the working directory
mv ~/Downloads/filename.zip ./workDir

# first term specifies command/program to run
# - specifies additional options
# x specifies extract the following
# v specifies verbose, give details while executing
# f specifies file, will not work without
tar -xvf filename.zip

# 2. Initial data lookover
- https://casadocs.readthedocs.io/en/latest/notebooks/data_examination.html

In [ ]:
# open CASA in working directory
casa

# list individual scans including data, date, scans, sources (fields), antenna locations, etc
# IntegrationTime is total duration
# identify an antenna close to the centre with no issues for calibration
listobs(vis=filename, listfile=filename[:-2]+'txt')

# examine the observing conditions by opening the observer log .obslog
# VLA observer logs can be downloaded from http://www.vla.nrao.edu/cgi-bin/oplogs.cgi
# enter the date range of interest and click Show Logs

# get a sense of the array by plotting
plotants(vis=filename, figfile=antennaLayoutPlot, exclude=badAntenna)
clearstat() # This removes any existing table locks generated by plotants()


# 3. Examine and edit data

In [ ]:
# @title flag known bad data
# generally good to plot data as time and frequency series to look for flags
# other options are raw data, model, caltables, calibrated data
# flags are stored as a binary column in the measurement set (.ms)
# flagbackup makes a separate file of previous flag states, on by default
# ms file likely has some flags already included from pre processing

# flag the first scan if it is a setup scan
# flagbackup=True: saves a copy of existing flags before entering any new flags, up to preference, # true by default
# mode='manual': specific data will be selected for editing
# scan='1': selects the first scan, must be a string
flagdata(vis=filename, flagbackup=True, mode='manual', scan='1')

# from the .obslog file (open in text editor)
flagdata(vis=filename,flagbackup=True, mode='manual',antenna=badAntenna)

# the array needs to settle down when beginning a scan
# flag the initial samples at the start of each scan, known as quack flagging
# mode='quack' : Quack is another mode in which the same edit will be applied to all scans for all baselines
# quackmode='beg' : In this case, data from the start of each scan will be flagged. Other options include flagging data at the end of the scan.
# quackinterval=sampleTime : In this data set, the sampling time is sampleTime seconds, so this choice flags the first sample from all scans on all baselines.
flagdata(vis=filename,mode='quack',quackinterval=float(sampleTime),quackmode='beg')
flagdata()


In [ ]:
# @title examine data for other obvious problems

# plotms() normalizes amplitude
# selectdata=True : One can choose to plot only selected subsets of the data.
# correlation='RR,LL' : Plot only the left- and right-handed polarization products. The cross-terms ('RL' and 'LR') will be close to zero for non-polarized sources.
# averagedata=True: One can choose to average data points before plotting them.
# avgchannel=str(channel) : With this plot, we are mainly interested in the fields vs time. Averaging over all channels in the spectral window makes the plotting faster.
# coloraxis='field' : Color-code the plotting symbols by field name/number.
clearstat()
plotms(vis=filename, selectdata=True, correlation='RR,LL', averagedata=True, avgchannel=str(channels), coloraxis='field', plotfile=ampTimePlot, overwrite=True)

FIX # confirm refantenna doesnt drop out
# plot all baselines involving antenna ant1
# this shows when various antennas drop out, assuming ant1 never does
# antennas with missing blocks should not be chosen as the reference calibrator
plotms(vis=filename, field='', correlation='', timerange='', antenna='C01', spw='', xaxis='time',
       yaxis='antenna2', plotrange=[-1,-1,0,26], coloraxis='field', plotfile=datastream2, overwrite=True)


# 4. Calibrate the data

In [ ]:
# @title correct antenna positions
# see the corrections reported in the logger
# Online databse for baseline corrections
gencal(vis=filename,caltable=antposTable ,caltype='antpos')


In [ ]:
# @title initial flux density scaling

# determine antenna gains by comparing observed flux density of a calibrator to its known flux density
# find all the available calibrator models with setjy
# setjy() sets the model column, with default amp of 1 and phase of 0
setjy(vis=filename, listmodels=True)

# use the C band VLA standard model for 3C286
# setjy should output IQUV flux density, then I flux density for each channel
# setjy returns a dict of reference flux density, usually not needed
# field=ampCal: specify the flux density calibrator
# standard='Perley-Butler 2017': use the most recent flux density scale
# model=model: compensate for slight resolution effects (deviations from point source model)
# usescratch=False: ASA saves the model information and calculates individual model visibilities on-the-fly when needed to save disk space
# scalebychan=True: let setjy determine a flux density value per channel rather than one value for the entire spectral window
# spw=' ': we have isolated spectral window 0, thus no need to specify spw
setjy(vis=filename,field=ampCal,standard='Perley-Butler 2017', model=model,usescratch=False,scalebychan=True,spw='')


In [ ]:
# @title initial phase calibration

# solve for phase variations with time to prevent decorrelation when vector averaging for the final bandpass solution, use all calibrator fields
# gaincal() compares the data column to model column and determines the solution that transforms data to model
# caltable=phaseTable: gain solutions stored here
# field=ampCal,gainCal,polCal: specify the calibrators
# refant=refAnt: reference antenna near the centre of the array
# spw=spwMid: chosen subset of the channels from which gain corrections are determined, should be the central 10% of the channels in the band
# gaintype='G': compute the complex gain solutions, one per antenna per spw per polarization per solution interval, default V stokes is zero
# calmode='p': solve for only the phase portion of the gain.
# solint='int': solution interval, 'int' refers to a single integration time (sampleTime)
# minsnr=5: restrict the solutions to be at relatively high signal-to-noise ratios
# gaintable=[antposTable]: apply antenna position corrections
gaincal(vis=filename, caltable=phaseTableAll, field=','.join((ampCal,gainCal,polCal)), refant=refAnt, spw=spwMid, gaintype='G', calmode='p', solint='int', minsnr=5)#, gaintable=[antposTable])

# iterate through each antenna, click the green arrows
plotms(vis=phaseTableAll, xaxis='time', yaxis='phase', coloraxis='corr', iteraxis='antenna', plotrange=[-1,-1,-180,180],
       gridrows=gridr, gridcols=gridc, width=w, height=h, exprange='all', overwrite=True, plotfile=gainphaseRLTimeAllPlot)

# flag antenna ea05 for displaying phase jumps
flagdata(vis=filename, flagbackup=True, mode='manual', antenna='ea05')

# obtain calibration table used for solving bandpass solution
gaincal(vis=filename, caltable=phaseTable, field=ampCal, refant=refAnt , spw=spwMid, calmode='p', solint='int', minsnr=5, gaintable=[antposTable])

# (O) plot R,L for the first block of data
plotms(vis=phaseTable, xaxis='time', yaxis='phase', coloraxis='corr', field=ampCal, iteraxis='antenna', plotrange=[-1,-1,-180,180], timerange=timeRange,
       gridrows=gridr, gridcols=gridc, width=w, height=h, exprange='all', overwrite=True, plotfile=gainphaseRLTimePlot)


In [ ]:
# @title delay calibration

# solve for relative delays of each antenna to a reference antenna
# field=ampCal : For the bandpass calibrator
# refant=refAnt : Delays will be relative to this antenna, make sure it is there!
# spw=spwWide : Widest possible frequency range in the spw, avoiding edge channels because they have lower sensitivity
# gaintype='K' : Compute K (delay) solutions, one per antenna per spw per polarization per solution interval
# solint='inf',combine='scan' : Only need one solution averaged over all times and scans. # solint='inf ' sets the solution interval to 'infinite' but respects scan boundaries
# combine='scan' combines data across scan boundaries
# minsnr=5 : Restrict the solutions to be at relatively high signal-to-noise ratios
# gaintable=[antposTable,phaseTable] : Use the antpos and G0 tables
gaincal(vis=filename, caltable=delayTable, field=ampCal, refant=refAnt, spw=spwWide, gaintype='K', solint='inf', combine='scan', minsnr=5, gaintable=[antposTable,phaseTable])

# plot solutions in nanoseconds as a function of antenna
plotms(vis=delayTable, xaxis='antenna1', yaxis='delay', coloraxis='baseline', plotfile=delayAntennaPlot)


In [ ]:
# @title bandpass calibration

# correct for the complex bandpass
# all VLA data is in spectral line mode, requiring a bandpass solution to account for gain variations with frequency
# sensitivity is usually less on the edges of the band due to common receiver engineering
# sensitivity fluctuations across band vary by individual receivers
# bandpass solution should be continuous from physical principles

# plot RR correlated data to motivate a bandpass solutions
# visibility data amplitude is ideally constant with frequency
# variations in amplitude with frequency reflect different antenna bandpasses
# plotms(vis=filename, x-axis='chan', y-axis='amp', field=ampCal, antenna='ea01', correlation='RR', plotrange=[-10,70,0,.24], coloraxis='antenna2')

# field=ampCal: the amplitude calibrator is suitable as the bandpass calibrator
# solint='inf': solution interval, 'inf' combines solutions for all times since the observation contains multiple scans on the bandpass calibrator
# combine='scan': averages over all scans, by default derives a bandpass correction for each scan (which may be desirable for high dynamic range spectral lines)
# bandtype='B': derives bandpass solution on a channel-by-channel basis, 'BPOLY' alternatively fits an n-th order polynomial
# gaintable=[antposTable, phaseTable, delayTable]: calibrates for antenna position, phase gain, delay gain, before computing bandpass
bandpass(vis=filename, caltable=bandpassTable, field=ampCal, spw='', refant=refAnt, combine='scan', solint='inf', bandtype='B', gaintable=[antposTable,phaseTable,delayTable])

# display bandpass solutions
# expect bandpass phases to be constant with frequency due to phase/delay calibration
plotms(vis=bandpassTable, field=ampCal, xaxis='chan', yaxis='amp', coloraxis='corr', iteraxis='antenna',
       gridrows=gridr, gridcols=gridc, width=w, height=h, exprange='all', overwrite=True, plotfile=bandpassGainAmpChannelPlot)

plotms(vis=bandpassTable, field=ampCal, xaxis='chan', yaxis='phase', coloraxis='corr', plotrange=[-1,-1,-180,180], iteraxis='antenna',
       gridrows=gridr, gridcols=gridc, width=w, height=h, exprange='all', overwrite=True, plotfile=bandpassGainPhaseChannelPlot)


In [ ]:
# @title gain calibration

# correct for the complex amplitude and phase gains
# this 2nd round of amplitude/phase calibration includes a source closer to the target to minimise atmospheric effects from differing lines of sight
# observe the gain calibrator gainCal
# CASA tutorial refers to it as the phase calibrator since on the timescale of hours, we expect phase to change much more than amplitude

# first rederive the amplitude and phase gains for the flux density calibrator (ampCal)
gaincal(vis=filename, caltable=gainTable, field=ampCal,spw=spwWide, solint='inf', refant=refAnt, gaintype='G',
        calmode='ap', solnorm=False, gaintable=[antposTable,delayTable,bandpassTable], interp=['linear','linear','nearest'])

# next derive the amplitude and phase gains for the near-target phase calibrator
gaincal(vis=filename, caltable=gainTable, field=gainCal, spw=spwWide, solint='inf', refant=refAnt, gaintype='G',
        calmode='ap', gaintable=[antposTable,delayTable,bandpassTable], append=True)

# repeat for the polarization calibrator polCal
gaincal(vis=filename,caltable=gainTable, field=polCal, spw=spwWide, solint='inf', refant=refAnt, gaintype='G',
        calmode='ap', gaintable=[antposTable,delayTable,bandpassTable], append=True)

# plot the amplitude and phase gain solutions
# expect smooth solutions for each antenna with time
plotms(vis=gainTable, xaxis='time', yaxis='phase', gridrows=1, gridcols=2, iteraxis='corr', coloraxis='baseline', plotrange=[-1,-1,-180,180], plotfile=gainPhaseVsTimePlot)
plotms(vis=gainTable, xaxis='time', yaxis='amp', gridrows=1, gridcols=2, iteraxis='corr', coloraxis='baseline', plotfile=gainAmpVsTimeplot)

# plot the complex polarization ratio
# this checks that the chosen reference antenna has good phase stability, the phase difference between R and L polarization is stable with time
# a few degrees of drift but no phase jumps is acceptable
plotms(vis=gainTable, xaxis='time', yaxis='phase', correlation='/', coloraxis='baseline', plotrange=[-1,-1,-180,180], plotfile=complexPolarizationRatioPlot)

In [ ]:
# @title polarization calibration, generating model
# without a good polarization calibrator, we are limited by how well we know the polarization of the other calibrators

# setjy() calculates Stokes Q and U from reference frequency, Stokes I, polarization fraction, and polarization angle
# inspect casa-<timestamp>.log for the previous call to setjy() to find the values of Stokes I
# compute the spectral index
alpha = log(i1/i0)/log(float(refFreq1[:-3])/float(refFreq0[:-3]))

# generate the polarization model
# Stokes V has been set to 0, corresponding to no circular polarization
setjy(vis=filename, field=ampCal, standard='manual', spw='0', fluxdensity=[i0,0,0,0], spix=[alpha,0],
      reffreq=refFreq0, polindex=[c0,0], polangle=[d0,0], scalebychan=True, usescratch=False)

# plot model RR amplitude vs channel
plotms(vis=filename, field=ampCal, correlation='RR', timerange=timeRange, antenna=baseline0, xaxis='channel', yaxis='amp', ydatacolumn='model',
       plotfile=modelRRPolAmpVsChannelPlot, overwrite=True)

# plot model RL amplitude vs channel
plotms(vis=filename, field=ampCal, correlation='RL', timerange=timeRange, antenna=baseline0, xaxis='channel', yaxis='amp', ydatacolumn='model',
       plotfile=modelRLAmpVsChannelPlot, overwrite=True)

# plot model RL phase difference vs channel
# expect model RL phase difference to be constant with frequency
plotms(vis=filename, field=ampCal, correlation='RL', timerange=timeRange, antenna=baseline0, xaxis='channel', yaxis='phase', ydatacolumn='model', plotrange=[-1,-1,-180,180],
       plotfile=modelRLPhaseVsChannelPlot, overwrite=True)


In [ ]:
# @title polarization calibration, solving for cross hand delays (RL, LR)
# subset of delay calibration
# RL LR wires into correlator instead of LL RR
# polarized calibrators are ideal

# calculate cross hand delay calibration table
gaincal(vis=filename, caltable=crossDelayTable, field=ampCal, spw=spwWide, gaintype='KCROSS', solint='inf', combine='scan',
        refant=refAnt, gaintable=[antposTable, delayTable, bandpassTable, gainTable], gainfield=['','','', ampCal], parang=True)

# plot cross hand delay calibration table
plotms(vis=crossDelayTable, xaxis='antenna1', yaxis='delay', coloraxis='corr', plotfile=crossDelayTablePlot, overwrite=True)


In [ ]:
# @title polarization calibration, solving for leakage terms
# unpolarized calibrators are ideal as any leakage must be systemic

# polcal uses the Stokes I, Q, and U values in the model data (Q and U being zero for our unpolarized calibrator) to derive the leakage solutions
polcal(vis=filename, caltable=leakageTable0, field=polCal, spw=spwWide, refant=refAnt, poltype='Df', solint='inf', combine='scan',
       gaintable=[antposTable, delayTable, bandpassTable, gainTable, crossDelayTable], gainfield=['','','',polCal,''])

# plot leakage amplitude and phase vs channel
plotms(vis=leakageTable0, xaxis='chan', yaxis='amp', iteraxis='antenna', coloraxis='corr',
       gridrows=gridr, gridcols=gridc, width=w, height=h, exprange='all', overwrite=True, plotfile=leakageAmpVsChannelPlot)
plotms(vis=leakageTable0, xaxis='chan', yaxis='phase', iteraxis='antenna', coloraxis='corr', plotrange=[-1,-1,-180,180],
       gridrows=gridr, gridcols=gridc, width=w, height=h, exprange='all', overwrite=True, plotfile=leakagePhaseVsChannelPlot)

# plot leakage amplitude vs antenna index
plotms(vis=leakageTable0, xaxis='antenna1', yaxis='amp', coloraxis='corr', overwrite=True,  plotfile=leakageAmpVsAntennaPlot0)

# (TUTORIAL)
# rescue an antenna that was missing for scans on the polarization calibrator
# by observing the gain calibrator for sufficient parallactic angle (> 60 deg, a few hours), it can be used as a calibrator of unknown polarization
polcal(vis=filename, caltable=leakageTable, field=gainCal, spw=spwWide, refant=refAnt, poltype='Df+QU', solint='inf', combine='scan',
       gaintable=[antposTable, delayTable, bandpassTable, gainTable, crossDelayTable], gainfield=['','','',gainCal,''])

# (TUTORIAL)
# plot leakage amplitude vs antenna index again
# now ea04 (index 3) is present
# this plot looks similar to the previous plot, suggesting that our method is good since it produces similar results even with different calibrators
plotms(vis=leakageTable, xaxis='antenna1', yaxis='amp', coloraxis='corr', overwrite=True,  plotfile=leakageAmpVsAntennaPlot)


In [ ]:
# @title polarization calibration, solving for the RL polarization angle
# polarized calibrators are ideal as they have a known polarization angle

# calibrate the RL phase
polcal(vis=filename, caltable=polPhaseTable, field=ampCal, combine='scan', refant=refAnt, poltype='Xf', solint='inf',
       gaintable=[antposTable, delayTable, bandpassTable, gainTable, crossDelayTable, leakageTable], gainfield=['','','',ampCal,'',''])

# plot position angle gain phase vs channel
# we expect this plot to be continuous based on physical principles
plotms(vis=polPhaseTable, xaxis='chan', yaxis='phase', overwrite=True, plotfile=polGainPhaseVsChannelPlot)


In [ ]:
# @title scaling the Amplitude Gains
# the relative amplitudes are scaled to absolute amplitudes in this step
# this is useful for comparing to known values/other observations

# produce a new calibration table containing properly-scaled amplitude gains for the secondary calibrator
myscale = fluxscale(vis=filename, caltable=gainTable, fluxtable=fluxTable, reference=[ampCal], transfer=[gainCal,ampCal], incremental=False)

# plot the rescaled amplitudes vs time
# expect less variation over time, showing amplitude gain factors are now more similar compared to the plots right after gain calibration
plotms(vis=fluxTable,xaxis='time',yaxis='amp',correlation='R',coloraxis='baseline')
plotms(vis=fluxTable,xaxis='time',yaxis='amp',correlation='L',coloraxis='baseline')


# 5. Applying the calibration

In [ ]:
# @title applying the calibration

# apply the calibrations to each calibrator
applycal(vis=filename, field=ampCal, gaintable=[antposTable,fluxTable,delayTable,bandpassTable,crossDelayTable,leakageTable,polPhaseTable],
         gainfield=['',ampCal,'','','','',''], interp=['','nearest','','','','',''], calwt=[False], parang=True)
applycal(vis=filename, field=polCal, gaintable=[antposTable,fluxTable,delayTable,bandpassTable,crossDelayTable,leakageTable,polPhaseTable],
         gainfield=['',polCal,'','','','',''], interp=['','nearest','','','','',''], calwt=[False], parang=True)
applycal(vis=filename, field=gainCal, gaintable=[antposTable,fluxTable,delayTable,bandpassTable,crossDelayTable,leakageTable,polPhaseTable],
         gainfield=['',gainCal,'','','','',''], interp=['','nearest','','','','',''], calwt=[False], parang=True)

# apply the calibrations to the target source
applycal(vis=filename, field=mosaic, gaintable=[antposTable,fluxTable,delayTable,bandpassTable,crossDelayTable,leakageTable,polPhaseTable],
         gainfield=['',gainCal,'','','','',''], interp=['','linear','','','','',''], calwt=[False], parang=True)


In [ ]:
# @title plot the corrected calibrator sources
# bad data that requires flagging and recalibration may show up here
# we expect amplitude to decrease with frequency, linearly for our relatively narrow band
# this can be due to scattering, absorption, and receiver design

# plot corrected amplitude and phase vs channel for the amplitude calibrator
plotms(vis=filename, field=ampCal, correlation='', timerange=timeRange, antenna='', avgtime=avgTime, xaxis='channel', yaxis='amp', ydatacolumn='corrected', coloraxis='corr',
       overwrite=True, plotfile=ampCalCorrectedAmpVsChannelPlot)
plotms(vis=filename, field=ampCal, correlation='', timerange=timeRange, antenna='', avgtime=avgTime, xaxis='channel', yaxis='phase', ydatacolumn='corrected', coloraxis='corr',
       overwrite=True, plotfile=ampCalCorrectedPhaseVsChannelPlot, plotrange=[-1,-1,-180,180])

# plot the corrected amplitude and phase vs channel for the gain calibrator
# coloraxis() was not in the tutorial
plotms(vis=filename, field=gainCal, correlation='RR,LL', timerange='', antenna='', avgtime=avgTime, xaxis='channel', yaxis='amp', ydatacolumn='corrected', coloraxis='corr'
       overwrite=True, plotfile=gainCalCorrectedAmpVsChannelPlot)
plotms(vis=filename, field=gainCal, correlation='RR,LL', timerange='', antenna='', avgtime=avgTime, xaxis='channel', yaxis='phase', ydatacolumn='corrected', coloraxis='corr',
       overwrite=True, plotfile=gainCalCorrectedPhaseVsChannelPlot, plotrange=[-1,-1,-180,180])

# plot the corrected amplitude vs corrected phase for RR or LL for the unpolarized gain calibrator
# expect an oval of visibilities, with some scatter, centered at zero phase and at the amplitude
# see tutorial version 4.7
# plotms(vis=filename, field=gainCal, correlation='RR,LL', timerange='',antenna='', avgtime=avgTime, xaxis='phase', xdatacolumn='corrected', yaxis='amp', ydatacolumn='corrected',
#        plotrange=[-180,180,0,3], coloraxis='corr')

In [ ]:
# @title obtain the calibrated measurement set

# split off the targets and create a new, calibrated measurement set
split(vis=filename, outputvis=filenameCal, datacolumn='corrected', field=mosaic)

# correct the data weights (weight and sigma columns) in the measurement set
# remove the effects of relative noise scatter that may have been introduced from flagging uneven bits in the visibility data between the channels and times
statwt(vis=filenameCal, datacolumn='data')

# 6. Initial imaging

In [ ]:
# @title plot calibrated amplitude vs baseline
# we expect greater observed flux at shorter baselines for extended sources, since shorter baselines see more of the sky, supernovae remnants are extended
# we expect point sources to be at constant flux with frequency
# field numbers have changed
plotms(vis=filenameCal, xaxis='uvwave', yaxis='amp', ydatacolumn='data', field=target, avgtime=avgTime/2, correlation='RR',
        overwrite=True, plotfile=targetCorrectedAmpVsBaselinePlot)


In [ ]:
# @title image and deconvolute the calibrated target

# tclean() only removes the contributions of the dirty beam
# noise part of the true sky image (residuals) are retained
# VLA resolves 1/16k radians on longest baseline which is ~12 arcsec
# effective cleaning occurs at 3-5 image pixels per resolution pixel (12"/2.5"~5)
# target is ~ 9' ~ 540" ~ 216 image pixels
# efficient if the image dimensions are a multiple of 30
# tclean() applies an internal 1.2 multipler on image dimension
# use M2 to select regions containing emission sources
# double M2 to confirm region
# select Erase instead of Add to select regions to remove
# tap M3 to change saturation
# iterate multi scale clean
tclean(vis=filenameCal,
      imagename=cleanImage,
      field='',spw='',
      mode='mfs',
      niter=5000,
      gain=0.1, threshold='1.0mJy',
      psfmode='clark',
      imagermode='mosaic', ftmachine='mosaic',
      multiscale=[0],
      interactive=True,
      imsize=[480,480], cell=['2.5arcsec','2.5arcsec'],
      stokes='I',
      weighting='briggs',robust=0.5,
      usescratch=False)

In [ ]:
# @title view the target image
# comprised of clean components convolved with a restoring beam and added to the remaining residuals
viewer(cleanImage+'.image')

# Bugs & Other
- screen locked, just showing background, run the following in Terminal
 - ssh -L 5903:localhost:5900 hanyu@keppie.dunlap.utoronto.ca
 - % kill WindowServer
 - % killall -u hanyu
- plotting the 4 correlations of 25 baselines per antenna1 gives 100 lines
 - corr='RR' only plots the RR correlation, giving only 25 lines
 - plotms(vis=filename, xaxis='freq', yaxis='amp', iteraxis='antenna',field=ampCal)
- radio frequency interference (RFI) from human sources needs to be filtered
 - astronomical sources are very faint, thus flag strong signals that appear when inspecting RL, LR fields
 - external programs and CASA can do this much more precisely than by eye
